In [ ]:
import time
import torch
import triton
import triton.language as tl
import torch.nn.functional as F

In [ ]:
sample=torch.tensor(
    [
        [1,2,3],
        [5,6,7]
    ],
    dtype=torch.float32,
    device='cuda'
)

out=F.softmax(
    sample,
    dim=1
)
out

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]], device='cuda:0')

In [ ]:
def softmax(
        x:torch.Tensor
)-> torch.Tensor:
    x_max=x.max(dim=1)[0]
    safe_x=x-x_max[:,None]

    output=torch.exp(
        safe_x
    )
    output=output/output.sum(dim=1)[:,None]
    
    return output

In [ ]:
a=time.time()
softmax(sample)
time.time()-a

0.0009968280792236328

In [ ]:
@triton.jit
def forward( output_ptr,stride_output_row,input_ptr,
    stride_input_row,
    num_columns,
    block_size: tl.constexpr
    ):
    row_index=tl.program_id(0)
    row_start_ptr=input_ptr+(row_index*stride_input_row)
    col_offsets=tl.arrange(0,block_size)
    row=row_start_ptr+col_offsets
    


x = torch.tensor([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]])
x.stride()
(5, 1)
x.stride(0)
5
x.stride(-1)
1


In [ ]:
def softmax_triton(x:torch.Tensor)-> torch.Tensor:
    rows,columns=x.shape
    block_size=triton.next_power_of_2(columns)
    num_warps=4
    print(block_size)

    sm_out=torch.empty_like(x)


    forward[(rows,)](
        sm_out,
        sm_out.stride(0),
        x,
        x.stride(0),
        columns,
        block_size,
        num_warps
    )     


NameError: name 'torch' is not defined

In [ ]:
torch.Tensor(torch.rand(1023))

tensor([0.6621, 0.5350, 0.5823,  ..., 0.1118, 0.7255, 0.0086])

In [ ]:
softmax_triton()

TypeError: softmax_triton() missing 1 required positional argument: 'x'